## AMP - Modelling

First we will import all dependencies.

In [11]:
import os
import pandas as pd

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
# To visualize pipeline diagram - 'text', or 'diagram'
from sklearn import set_config
# Import XGBoost
from xgboost import XGBClassifier

random_state = 10

In [2]:
# Import the script from different folder
import sys  
sys.path.append('../../scripts')

import file_utilities as fu
import modelling_utilities as mu

#### Set one of three project tasks (*acp*, *amp*, *dna_binding*)

In [3]:
# task - ['acp', 'amp', 'dna_binding']
task = 'amp'

### Define Pipelines

Define pipelines for ML algorithms.  
As preprocessing steps we will use `MinMaxScaler()` and `PCA()`.

In [4]:
# Define number of PCA components
num_pca_components = 1000

pipelines = {
    'xgb' : make_pipeline(MinMaxScaler(), 
                          PCA(num_pca_components),
                          XGBClassifier(random_state=random_state)),
    'lr' : make_pipeline(MinMaxScaler(),
                         PCA(num_pca_components),
                         LogisticRegression(max_iter=25000, random_state=random_state)),    
    'svm' : make_pipeline(MinMaxScaler(),
                          PCA(num_pca_components),
                          SVC(random_state=random_state)),
    'rf' : make_pipeline(MinMaxScaler(),
                         PCA(num_pca_components),
                         RandomForestClassifier(random_state=random_state))
}

### Define Hyperparameter Grids

Define hyperparamter grids for chosen ML algorithms.

In [5]:
# XGBoost
xgb_grid = {
        'xgbclassifier__max_depth': [3, 5],
         'xgbclassifier__n_estimators': [100, 200],
        }
# SVC
svm_grid = {
        'svc__kernel' : ['linear', 'rbf'],
        'svc__C': [0.01, 0.1, 1]
    }
# Random Forest
rf_grid = {
        'randomforestclassifier__n_estimators' : [100, 150],
        'randomforestclassifier__min_samples_leaf' : [1, 3],
        'randomforestclassifier__min_samples_split' : [2, 3]
    }
# Logistic Regression
lr_grid = {
        'logisticregression__C' : [0.1, 1],
        'logisticregression__solver' : ['lbfgs', 'saga']
    }

#### Create dictionary for hyperparameter grids

In [6]:
# Create hyperparameter grids dictionary
hp_grids = {
    'lr' : lr_grid,
    'svm' : svm_grid,
    'rf' : rf_grid,
    'xgb' : xgb_grid
}

### Get embedding folders and fasta files for the task.

For our modelling we will need to use previously created fasta files:

In [7]:
!tree -nhDL 1 ../../data/"{task}"/ -fP *.fa | grep fa

├── [216K Sep  3 22:34]  ../../data/amp/all_data.fa


<br>

and embedding `.pt` files in these folders:

In [8]:
!tree -nhDL 3 ../../data/"{task}"/ -df | grep 'esm1\|mt\|dlm'

│       ├── [4.0K Sep  6 17:55]  ../../data/amp/esm/all_data/amp_all_esm1b_mean
│       └── [4.0K Sep  6 17:46]  ../../data/amp/esm/all_data/amp_all_esm1v_mean
        ├── [4.0K Sep  4 17:39]  ../../data/amp/prose/all_data/amp_all_dlm_avg
        ├── [4.0K Sep  4 22:27]  ../../data/amp/prose/all_data/amp_all_dlm_max
        ├── [4.0K Sep  4 22:38]  ../../data/amp/prose/all_data/amp_all_dlm_sum
        ├── [4.0K Sep  4 22:56]  ../../data/amp/prose/all_data/amp_all_mt_avg
        ├── [4.0K Sep  4 23:05]  ../../data/amp/prose/all_data/amp_all_mt_max
        └── [4.0K Sep  4 23:16]  ../../data/amp/prose/all_data/amp_all_mt_sum


<br>  

To get paths for embedding folders and fasta files we will use the function `get_emb_folders()`.

In [9]:
pt_folders, fa_files = mu.get_emb_folders(task)

## Modelling Loop

The modelling loop includes the following steps:

1. Loop through embedding folders
2. Run the function `read_embeddings()` for all_data embeddings to get `X` and `y`
3. Split `X` and `y` into train and test sets
4. Define and print the output header
5. Use the function `fit_tune_CV()`to to do the following:
   - use above defined `pipelines` and `hp_grids` dictionaries and `GridSearchCV()` to get models
   - save the models with `joblib()`
   - create a dictionary of the models for one set of embeddings
6. Run the function `evaluation()` to create an evaluation dataframe for one set of embeddings


In [12]:
# Initialize dictionary to keep evaluation dataframes 
# One dataframe per embeddings folder (train+test, or all_data)
df_models = {}

for i in range(len(pt_folders)):
    
    # No test, train files, only one: all_data
    path_fa, path_pt = fa_files[0], pt_folders[i]
    pool = os.path.split(path_pt)[1].split('_')[-1]
    emb_layer = 33 if 'esm' in path_pt else 'layer'
    X, y, sequence_id_train = fu.read_embeddings(path_fa, path_pt, pool, emb_layer,print_dims=False)
    
    # Train-Test split
    # Split X and y into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,
                                                    random_state=random_state,
                                                   stratify=y)

    # Extensions for evaluations dataframes
    df_ext = os.path.split(path_pt)[1].split('_', 1)[1].split('_', 1)[1]
    
    # Printing output header
    ptm = df_ext.split('_')[0]
    ptr = 'ESM' if 'esm' in ptm else 'ProSE'
    print('-' * 75)
    print(f'\tPretrained Model "{ptm}" by {ptr} - Pooling Operation: "{pool}"')
    print('-' * 75)
    
    # Grid search and fit
    fitted_models = mu.fit_tune_CV(pipelines, hp_grids, 'accuracy', path_pt, X_train, y_train, task)
    
    # Save valuation dataframe into dictionary
    df_models[f'eval_{df_ext}'] = mu.evaluation(fitted_models, X_test, y_test)
  

---------------------------------------------------------------------------
	Pretrained Model "esm1b" by ESM - Pooling Operation: "mean"
---------------------------------------------------------------------------
esm1b_mean_xgb has been fitted and saved
esm1b_mean_lr has been fitted and saved
esm1b_mean_svm has been fitted and saved
esm1b_mean_rf has been fitted and saved
---------------------------------------------------------------------------
	Pretrained Model "esm1v" by ESM - Pooling Operation: "mean"
---------------------------------------------------------------------------
esm1v_mean_xgb has been fitted and saved
esm1v_mean_lr has been fitted and saved
esm1v_mean_svm has been fitted and saved
esm1v_mean_rf has been fitted and saved
---------------------------------------------------------------------------
	Pretrained Model "dlm" by ProSE - Pooling Operation: "avg"
---------------------------------------------------------------------------
dlm_avg_xgb has been fitted and saved


<br>

Let's list the keys of the `df_models` dictionary:

In [13]:
df_models.keys()

dict_keys(['eval_esm1b_mean', 'eval_esm1v_mean', 'eval_dlm_avg', 'eval_dlm_max', 'eval_dlm_sum', 'eval_mt_avg', 'eval_mt_max', 'eval_mt_sum'])

<br>
Let's check a dataframe for a randomly chosen key (set of embeddings)

In [14]:
import random
df_models[list(df_models.keys())[random.randint(0, len(df_models))]]

,cv_best_score,f1_macro,accuracy
model,,,
esm1v_mean_xgb,0.923127,0.918843,0.918892
esm1v_mean_lr,0.934676,0.933692,0.933729
esm1v_mean_svm,0.935005,0.934672,0.934718
esm1v_mean_rf,0.906302,0.910906,0.910979


## Collecting Evaluation Results into a DataFrame

To compare evaluations for all models, collect results from all dataframes into one dataframe.  

To do that, we will merge all dataframes from the dictionary `df_models`.

In [15]:
# Create dataframe with evaluations for all models

# initialize dataframe
eval_df_all = pd.DataFrame()
# concatenate all dataframes from dictionary df_models
# Iterate through all dictionary keys 
for i in df_models.keys():
    # Use a temporary dataframe to hold one iteraton's dataframe
    eval_df_t = df_models[i].copy().reset_index()
    eval_df_all = pd.concat([eval_df_all, eval_df_t])

# Set the column 'model' as an index
eval_df_all = eval_df_all.set_index('model')

#### Display the results sorted by "accuracy"

In [16]:
# Display the dataframe
eval_df_all.sort_values(by=['accuracy'], ascending=False)

,cv_best_score,f1_macro,accuracy
model,,,
mt_avg_lr,0.929726,0.944585,0.944609
mt_max_lr,0.931379,0.943593,0.943620
mt_avg_svm,0.931707,0.942601,0.942631
mt_sum_lr,0.929068,0.941597,0.941642
dlm_avg_svm,0.932037,0.940617,0.940653
mt_max_svm,0.933027,0.936669,0.936696
esm1v_mean_svm,0.935005,0.934672,0.934718
dlm_avg_lr,0.923787,0.934707,0.934718
dlm_sum_svm,0.926430,0.934672,0.934718


### Saving dataframe for future use

In [17]:
TASK = 'DBP' if task == 'dna_binding' else task.upper()
file_path = f'../../results/{TASK}_classifiers.csv'
eval_df_all.to_csv(file_path)

<br>

When you need to work with the results from that file, read it with the parameter `index_col=`:   
```python
df = pd.read_csv(file_path, index_col='model')
```